In [14]:
# # https://www.house.gov/representatives
# # I want to scrape the names of all the representatives from New York (state).

# import requests
# from bs4 import BeautifulSoup

# url = 'https://www.house.gov/representatives'
# response = requests.get(url)
# print(response.status_code)
# soup = BeautifulSoup(response.text, 'html.parser')
# # print(soup.prettify())

# #by-state > div > div > div.view-content (contains tables of representatives by state)
# # <caption id="state-alabama">Alabama </caption> (state looks like this)
# #housegov_reps_by_state-block_default-883106430 > tbody > tr:nth-child(1) is the first row of the table (a single representative from the first state)

# # I want to find the table that contains the representatives from New York

# # Locate the caption for New York using a text match
# ny_caption = soup.find('caption', string=lambda text: 'New York' in text)
# if ny_caption:
#     # Get the parent table of this caption
#     ny_table = ny_caption.find_parent('table')
#     if ny_table:
#         representatives = ny_table.find_all('tr')
        
#         # Print each representative's name assuming the name is in the first td
#         for rep in representatives[1:]:  # skip the header row if present
#             # for all td elements in the row, put all td elements in a list
#             tds = rep.find_all('td')
#             # strip and put into a list
#             print([td.text.strip() for td in tds])
#     else:
#         print("No table found for New York")
# else:
#     print("Couldn't find the New York section on the page")

200
['1st', 'LaLota, Nick', 'R', '1530 LHOB', '(202) 225-3826', 'Armed Services|Homeland Security|Small Business']
['2nd', 'Garbarino, Andrew', 'R', '2344 RHOB', '(202) 225-7896', 'Financial Services|Homeland Security|Ethics']
['3rd', 'Suozzi, Thomas R.', 'D', '1117 LHOB', '(202) 225-3335', 'Homeland Security']
['4th', "D'Esposito, Anthony", 'R', '1508 LHOB', '(202) 225-5516', 'House Administration|Homeland Security|Transportation and Infrastructure']
['5th', 'Meeks, Gregory', 'D', '2310 RHOB', '(202) 225-3461', 'Financial Services|Foreign Affairs']
['6th', 'Meng, Grace', 'D', '2209 RHOB', '(202) 225-2601', 'Appropriations']
['7th', 'Velazquez, Nydia', 'D', '2302 RHOB', '(202) 225-2361', 'Financial Services|Natural Resources|Small Business']
['8th', 'Jeffries, Hakeem', 'D', '2433 RHOB', '(202) 225-5936', '']
['9th', 'Clarke, Yvette', 'D', '2058 RHOB', '(202) 225-6231', 'Homeland Security|Energy and Commerce']
['10th', 'Goldman, Daniel', 'D', '245 CHOB', '(202) 225-7944', 'Select Subcom

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

url = 'https://www.house.gov/representatives'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup.prettify())

state_section = soup.find('a', {'href': '#by-state'})
if state_section is None:
    print("Couldn't find the section for states")
    exit()

# print(state_section)

# Find all caption elements which typically contain state names
captions = soup.find_all('caption')

# Prepare a list to hold all representative data
all_reps = []

# Iterate over each state's caption
for caption in captions:
    state = caption.get_text(strip=True)
    table = caption.find_parent('table')
    if table:
        headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]
        rows = table.find_all('tr')[1:]  # Skipping the header row

        for row in rows:
            cols = row.find_all('td')
            if len(cols) == len(headers):  # Ensuring each row has the correct number of columns
                rep_data = {'State': state}
                rep_data.update({headers[i]: cols[i].get_text(strip=True) for i in range(len(cols))})
                all_reps.append(rep_data)

# Convert the list of dictionaries into a DataFrame
df_reps = pd.DataFrame(all_reps)

# Split the Name column into separate First Name and Last Name columns
df_reps[['fist_name', 'last_name']] = df_reps['Name'].str.split(', ', expand=True)

# Drop the original Name column
df_reps.drop(columns=['Name'], inplace=True)

# Create a full_name column by concatenating first_name and last_name
df_reps['full_name'] = df_reps['fist_name'] + ' ' + df_reps['last_name']

# Split Committee assignments into separate columns
# find max number of committees for any representative
max_cols = df_reps['Committee Assignment'].str.split('|').apply(len).max()

# Create new columns for each committee
for i in range(max_cols):
    df_reps[f'committee_{i+1}'] = df_reps['Committee Assignment'].str.split('|').str[i]

# Drop the original Committee Assignment column
df_reps.drop(columns=['Committee Assignment'], inplace=True)

# Display the DataFrame
# print(df_reps)

# check if inputs folder exists
# if not, create it

DATA_PATH = '../../data'

if not os.path.exists(DATA_PATH+'/inputs'):
    os.makedirs(DATA_PATH+'/inputs')

# Save the DataFrame to a CSV file
df_reps.to_csv(DATA_PATH+'/inputs/house_committees.csv', index=False)


In [19]:
# url = "https://senate.gov/general/committee_assignments/assignments.htm"
# response = requests.get(url)
# print(response.status_code)

403
